In [36]:
import pandas as pd
import os
import plotly.graph_objects as go
import dash
from dash import dash_table, dcc, html

In [37]:
def load_csv_files_to_memory(directory):

    csv_files = {}
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, directory)
                csv_files[relative_path] = pd.read_csv(file_path, sep='\t')
    return csv_files

In [38]:
input_dir = "Results/"  
output_dir = "ResultsCSV/" 

csv_data = load_csv_files_to_memory(input_dir)

backdoor_success_list = []
clean_accuracy_drop_list = []

In [39]:
for relative_path, data in csv_data.items():
    essential_columns = ['poisoning_rate', 'backdoor_success_rate', 'clean_accuracy_after', 'clean_accuracy_drop']
    hyperparameter_columns = [col for col in data.columns if col not in essential_columns]
    
    folder_name = os.path.dirname(relative_path).replace(os.sep, '_')
    data['experiment'] = folder_name + '_' + data[hyperparameter_columns].astype(str).agg('_'.join, axis=1)
    
    backdoor_success_list.extend([
        {'experiment': row['experiment'], row['poisoning_rate']: row['backdoor_success_rate']}
        for _, row in data.iterrows()
    ])
    
    clean_accuracy_drop_list.extend([
        {'experiment': f"{row['experiment']}_rate_{row['poisoning_rate']}", 
         'clean_accuracy_drop': row['clean_accuracy_drop']}
        for _, row in data.iterrows()
    ])

In [40]:
backdoor_success_data = pd.DataFrame(backdoor_success_list)
clean_accuracy_drop_data = pd.DataFrame(clean_accuracy_drop_list)

backdoor_success_table = backdoor_success_data.pivot_table(
    index='experiment', aggfunc='first'
).reset_index()

clean_accuracy_drop_table = clean_accuracy_drop_data.pivot_table(
    index='experiment', aggfunc='first'
).reset_index()

In [41]:
backdoor_success_table_DF = pd.DataFrame(backdoor_success_table)
clean_accuracy_drop_table_DF = pd.DataFrame(clean_accuracy_drop_table)

In [42]:
backdoor_success_table_DF['Model'] = backdoor_success_table_DF['experiment'].str.split('_').str[0]

In [43]:
clean_accuracy_drop_table_DF['Model'] = clean_accuracy_drop_table_DF['experiment'].str.split('_').str[0]

In [44]:
backdoor_success_table_DF['Attack'] = backdoor_success_table_DF['experiment'].str.extract(r'BKDR-([^_]+)')

In [45]:
clean_accuracy_drop_table_DF['Attack'] = clean_accuracy_drop_table_DF['experiment'].str.extract(r'BKDR-([^_]+)')

In [46]:
backdoor_success_table_DF

,experiment,0.01,0.05,0.1,Model,Attack
0,CNN_CNN-BKDR-composite_1000,11.481481,53.333333,59.814815,CNN,composite
1,CNN_CNN-BKDR-composite_10000,93.333333,100.000000,100.000000,CNN,composite
2,CNN_CNN-BKDR-composite_24000,11.666667,5.185185,37.962963,CNN,composite
3,CNN_CNN-BKDR-distributed_1000,31.851852,61.481481,73.148148,CNN,distributed
4,CNN_CNN-BKDR-distributed_10000,91.111111,94.814815,97.592593,CNN,distributed
...,...,...,...,...,...,...
195,Whisper_Whisper-SD-BKDR-bckgrndLow_/kaggle/inp...,0.000000,96.851852,96.481481,Whisper,bckgrndLow
196,Whisper_Whisper-SD-BKDR-reverb_0.3_150,0.000000,0.370370,10.740741,Whisper,reverb
197,Whisper_Whisper-SD-BKDR-reverb_0.3_75,0.925926,65.925926,71.481481,Whisper,reverb
198,Whisper_Whisper-SD-BKDR-reverb_0.6_150,1.111111,1.111111,30.740741,Whisper,reverb


In [47]:
clean_accuracy_drop_table_DF

,experiment,clean_accuracy_drop,Model,Attack
0,CNN_CNN-BKDR-composite_10000_rate_0.01,8.666667,CNN,composite
1,CNN_CNN-BKDR-composite_10000_rate_0.05,4.500000,CNN,composite
2,CNN_CNN-BKDR-composite_10000_rate_0.1,1.833333,CNN,composite
3,CNN_CNN-BKDR-composite_1000_rate_0.01,1.833333,CNN,composite
4,CNN_CNN-BKDR-composite_1000_rate_0.05,3.166667,CNN,composite
...,...,...,...,...
595,Whisper_Whisper-SD-BKDR-reverb_0.6_150_rate_0.05,-1.833333,Whisper,reverb
596,Whisper_Whisper-SD-BKDR-reverb_0.6_150_rate_0.1,-1.333333,Whisper,reverb
597,Whisper_Whisper-SD-BKDR-reverb_0.6_75_rate_0.01,-2.833333,Whisper,reverb
598,Whisper_Whisper-SD-BKDR-reverb_0.6_75_rate_0.05,1.666667,Whisper,reverb


In [48]:
clean_accuracy_drop_table_DF.isnull().sum().sum()

0

In [49]:
backdoor_success_table_DF.isnull().sum().sum()

0

In [50]:
backdoor_success_table_DF

,experiment,0.01,0.05,0.1,Model,Attack
0,CNN_CNN-BKDR-composite_1000,11.481481,53.333333,59.814815,CNN,composite
1,CNN_CNN-BKDR-composite_10000,93.333333,100.000000,100.000000,CNN,composite
2,CNN_CNN-BKDR-composite_24000,11.666667,5.185185,37.962963,CNN,composite
3,CNN_CNN-BKDR-distributed_1000,31.851852,61.481481,73.148148,CNN,distributed
4,CNN_CNN-BKDR-distributed_10000,91.111111,94.814815,97.592593,CNN,distributed
...,...,...,...,...,...,...
195,Whisper_Whisper-SD-BKDR-bckgrndLow_/kaggle/inp...,0.000000,96.851852,96.481481,Whisper,bckgrndLow
196,Whisper_Whisper-SD-BKDR-reverb_0.3_150,0.000000,0.370370,10.740741,Whisper,reverb
197,Whisper_Whisper-SD-BKDR-reverb_0.3_75,0.925926,65.925926,71.481481,Whisper,reverb
198,Whisper_Whisper-SD-BKDR-reverb_0.6_150,1.111111,1.111111,30.740741,Whisper,reverb


In [51]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("backdoor success rate per experiment"),
    dash_table.DataTable(
        id='table',
        columns=[{"name": col, "id": col} for col in backdoor_success_table_DF.columns],
        data=backdoor_success_table_DF.to_dict('records'),
        sort_action="native", 
    )
])

app.run_server(debug=False)

In [52]:
output_file = os.path.join(output_dir, 'RI_visuals.xlsx')

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    backdoor_success_table_DF.to_excel(writer, sheet_name='Backdoor Success Rate', index=False)
    clean_accuracy_drop_table_DF.to_excel(writer, sheet_name='Clean Accuracy Drop', index=False)